<a href="https://colab.research.google.com/github/taku369/tf-tutorial-2/blob/master/tensorflow_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices((dict(train), train_y))
for i, x in enumerate(ds_tensors):
  print(x)
  if i > 10:
    break

({'SepalLength': <tf.Tensor: shape=(), dtype=float64, numpy=6.4>, 'SepalWidth': <tf.Tensor: shape=(), dtype=float64, numpy=2.8>, 'PetalLength': <tf.Tensor: shape=(), dtype=float64, numpy=5.6>, 'PetalWidth': <tf.Tensor: shape=(), dtype=float64, numpy=2.2>}, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
({'SepalLength': <tf.Tensor: shape=(), dtype=float64, numpy=5.0>, 'SepalWidth': <tf.Tensor: shape=(), dtype=float64, numpy=2.3>, 'PetalLength': <tf.Tensor: shape=(), dtype=float64, numpy=3.3>, 'PetalWidth': <tf.Tensor: shape=(), dtype=float64, numpy=1.0>}, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
({'SepalLength': <tf.Tensor: shape=(), dtype=float64, numpy=4.9>, 'SepalWidth': <tf.Tensor: shape=(), dtype=float64, numpy=2.5>, 'PetalLength': <tf.Tensor: shape=(), dtype=float64, numpy=4.5>, 'PetalWidth': <tf.Tensor: shape=(), dtype=float64, numpy=1.7>}, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
({'SepalLength': <tf.Tensor: shape=(), dtype=float64, numpy=4.9>, 'SepalWidth': <tf.Tensor: sh

In [ ]:
# Create input functions
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating
    pytorchのdatasetのgetitemに相当
    featuresはpd.series (dataframeの行)
    labelsは？
    """
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)  # これもdataset型

In [ ]:
# Define the feature columns
print(train.keys())
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')


In [ ]:
# Instantiate an estimator
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpeomf7_uh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),  # ?? なぜlambda？->引数を指定できないから
    steps=5000)
# tmpに結果を保存

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-02T15:53:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpeomf7_uh/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.29303s
INFO:tensorflow:Finished evaluation at 2020-06-02-15:53:41
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7, average_loss 

In [ ]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def inf_input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: inf_input_fn(predict_x))

In [ ]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpeomf7_uh/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (76.9%), expected "Setosa"
Prediction is "Setosa" (36.3%), expected "Versicolor"
Prediction is "Virginica" (36.0%), expected "Virginica"


## 最小構成

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
x = np.random.randn(1000) * 10
y = np.random.randn(1000) * 5
label_x = (x > 0).astype(int)
label_y = (y > 0).astype(int)

In [ ]:
train = np.stack([x, y], axis=1)
label = np.stack([label_x, label_y], axis=1)
print(label[:10])

[[0 0]
 [1 1]
 [1 1]
 [0 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [0 1]
 [1 1]]


In [ ]:
print(x[:10])
print(y[:10])
print(label_x[:10])
print(label_y[:10])

[-21.46786541  16.3901866    4.75342577  -9.9698129   19.16357888
   4.02222608   0.8868771   11.50112749  -1.36971506  16.90391417]
[-2.39731738  5.9664881   7.47485038  3.92911625 -1.92755783 -3.25265554
  0.61416839  6.60598356  2.27262521  2.21812999]
[0 1 1 0 1 1 1 1 0 1]
[0 1 1 1 0 0 1 1 1 1]


In [ ]:
# 各列の前処理の定義
x_col = tf.feature_column.numeric_column("x_col", shape=(1,), normalizer_fn=lambda x: x / 10)
y_col = tf.feature_column.numeric_column("y_col", shape=(1,), normalizer_fn=lambda x: x / 5)
feature_cols = [x_col, y_col]

In [ ]:
# データセットの作成
# from_tensor_slicesはデータ構造そのままに値をtensorに変えられる感じ
# dictを入れればvalueがtensorになるし, tupleをいれればtensorのtupleになる
def input_fn(features, labels, training=True, batch_size=5):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)  # これもdataset型

In [ ]:
batch_size = 5
epoch = 10
data_num = len(x)
steps_per_epoch = np.ceil(data_num / batch_size)

In [ ]:
# モデルアーキテクチャ
decay_steps = 3 * steps_per_epoch
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps, 0.1, staircase=True)  # 3epochごとに減衰
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
head = tf.estimator.MultiLabelHead(2)
estimator = tf.estimator.DNNEstimator(head, [5], feature_cols, optimizer=optimizer)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbsq4ci5d', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# 学習
estimator.train(
    input_fn=lambda: input_fn({"x_col": x, "y_col": y}, label, training=True, batch_size=batch_size),
    steps=epoch * steps_per_epoch)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpbsq4ci5d/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.60379547, step = 0
INF

In [ ]:
eval_result = estimator.evaluate(input_fn=lambda: input_fn({"x_col": x[:10], "y_col": y[:10]}, label[:10], training=False, batch_size=batch_size))
print(eval_result)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-23T15:18:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbsq4ci5d/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.40191s
INFO:tensorflow:Finished evaluation at 2020-06-23-15:18:58
INFO:tensorflow:Saving dict for global step 2000: auc = 1.0, auc_precision_reca

In [ ]:
# 推論
print(x[:20])
predictions = estimator.predict(input_fn=lambda: tf.data.Dataset.from_tensor_slices({"x_col": x[:20], "y_col": y[:20]}).batch(1))
for pred in predictions:
  print(pred)

[-21.46786541  16.3901866    4.75342577  -9.9698129   19.16357888
   4.02222608   0.8868771   11.50112749  -1.36971506  16.90391417
   3.48503549  19.81610138   5.1662209    6.14844221  -6.24419541
  -6.15980765   3.6516977    1.10719437 -12.09730078   8.48242769]
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbsq4ci5d/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Don

## テンソル

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [ ]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


In [ ]:
x = tf.random.uniform([3, 3])

print("利用できるGPUはあるか: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("テンソルはGPU #0にあるか:  "),
print(x.device.endswith('GPU:0'))

print(x.device)

利用できるGPUはあるか: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
テンソルはGPU #0にあるか:  
True
/job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start
    
  print("10 loops: {:0.2f}ms".format(1000*result))

# CPUでの実行を強制
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# GPU #0があればその上での実行を強制
if tf.config.experimental.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # 2番めのGPUなら GPU:1, 3番目なら GPU:2 など
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 353.11ms
On GPU:
10 loops: 0.76ms


In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# CSVファイルを作成
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

In [ ]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [ ]:
print('ds_tensors の要素:')
for x in ds_tensors:
  print(x)

print('\nds_file の要素:')
for x in ds_file:
  print(x)

ds_tensors の要素:
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([ 9 25], shape=(2,), dtype=int32)
tf.Tensor([36 16], shape=(2,), dtype=int32)

ds_file の要素:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)


## カスタムレイヤ

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
    
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape=[int(input_shape[-1]), 
                                           self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)
  
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

Instructions for updating:
Please use `layer.add_weight` method instead.
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.04251426, -0.34367457, -0.33987272, -0.39539674,  0.55485445,
        -0.30297944,  0.21195811,  0.339792  , -0.5777545 ,  0.16114849],
       [-0.56565154,  0.5968016 , -0.04391193,  0.42725414,  0.5004844 ,
        -0.27212465, -0.06614321,  0.08520418,  0.40098685,  0.19749576],
       [-0.63188344,  0.3573261 ,  0.60468346,  0.17102188, -0.062316  ,
        -0.18814233, -0.40083164, -0.487683  , -0.44528878,  0.36336398],
       [-0.55297035, -0.12520415,  0

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)

    
block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0']


In [ ]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1), 
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>